In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [8]:
class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Sequential(
                      nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
                      #nn.BatchNorm2d(16),
                      nn.ReLU(inplace=True),
                      nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
                      #nn.BatchNorm2d(16),
                      nn.ReLU(inplace=True),
                      nn.MaxPool2d(kernel_size=2, stride=2))
      self.conv2 = nn.Sequential(
                      nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
                      #nn.BatchNorm2d(16),
                      nn.ReLU(inplace=True),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      #nn.Dropout(0.25)
                      )
      self.conv3 = nn.Sequential(
                      nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3,padding=1),
                      nn.BatchNorm2d(64),
                      nn.ReLU(inplace=True),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout(0.2))
      self.fc = nn.Sequential(
                      nn.Conv2d(64, 10, kernel_size=3),
                      nn.AdaptiveAvgPool2d((1, 1))
                      )

    
  def forward(self, x):
      x = self.conv1(x)
      x = self.conv2(x)
      x = self.conv3(x)
      x = self.fc(x)
      x = x.view((x.shape[0],-1))
      x = F.log_softmax(x, dim=1)
      return x

# class Net(nn.Module):
#   def __init__(self):
#       super(Net, self).__init__()
#       self.conv1 = nn.Conv2d(1, 32, 3, 1)
#       self.conv2 = nn.Conv2d(32, 64, 3, 1)
#       self.dropout1 = nn.Dropout2d(0.25)
#       self.dropout2 = nn.Dropout2d(0.5)
#       self.fc1 = nn.Linear(9216, 128)
#       self.fc2 = nn.Linear(128, 10)

#   def forward(self, x):
#       x = self.conv1(x)
#       x = F.relu(x)
#       x = self.conv2(x)
#       x = F.max_pool2d(x, 2)
#       x = self.dropout1(x)
#       x = torch.flatten(x, 1)
#       x = self.fc1(x)
#       x = F.relu(x)
#       x = self.dropout2(x)
#       x = self.fc2(x)
#       output = F.log_softmax(x, dim=1)
#       return output

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 16, 28, 28]           2,320
              ReLU-4           [-1, 16, 28, 28]               0
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           2,320
              ReLU-7           [-1, 16, 14, 14]               0
         MaxPool2d-8             [-1, 16, 7, 7]               0
            Conv2d-9             [-1, 64, 7, 7]           9,280
      BatchNorm2d-10             [-1, 64, 7, 7]             128
             ReLU-11             [-1, 64, 7, 7]               0
        MaxPool2d-12             [-1, 64, 3, 3]               0
          Dropout-13             [-1, 64, 3, 3]               0
           Conv2d-14             [-1, 1

In [10]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,), (0.5,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,), (0.5,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0941, Accuracy: 9698/10000 (97.0%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0543, Accuracy: 9818/10000 (98.2%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0199, Accuracy: 9938/10000 (99.4%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0189, Accuracy: 9946/10000 (99.5%)



loss=0.014330749399960041 batch_id=468: 100%|████████████████████████████████████████| 469/469 [00:21<00:00, 21.65it/s]



Test set: Average loss: 0.0206, Accuracy: 9938/10000 (99.4%)

